<div class="alert alert-block alert-info">
<span style="color: rgb(0,53,91);">
<center><img src="https://www.redage.org/sites/default/files/styles/large/public/img-logo-institucion/logo_iteso.jpg?itok=IzloBJS1" style="width:500px;height:142px;" title="Logo ITESO"></center>

<font face = "Times New Roman" size = "6"><b><center>Programacion para Mineria de Datos</center></b></font>
<font face = "Times New Roman" size = "5"><b><center>Model Evaluation</center></b></font>

<font face = "Times New Roman" size = "5"><b><center>Proyecto Final</center></b></font>
<div align="right"><font face = "Times New Roman" size = "2">nombres: Angel Aceves Manzo</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jorge Arturo Ponce Martin</font></div>
<div align="right"><font face = "Times New Roman" size = "2"> Jesus Vargas Pacheco</font></div>
</span></div>

In [1]:
import pandas as pd
import joblib, json
import numpy as np

In [2]:
##################################################################################################
Train = pd.read_csv("../01_preprocessing_results/preprocessing/T_train_final_objetivo.csv")
Test = pd.read_csv("../01_preprocessing_results/preprocessing/T_test_final_objetivo.csv")
##################################################################################################

In [3]:
X_train = Train.iloc[:, :-1]
y_train = Train.iloc[:, -1].to_numpy(dtype=float)

X_test = Test.iloc[:, :-1]
y_test = Test.iloc[:, -1].to_numpy(dtype=float)

In [4]:
modelo = joblib.load("../02_lineal_regression_results/regression_lineal/modelo_reg_lineal.pkl")

In [5]:
intercepto = modelo.named_steps["linreg"].intercept_
feature_names = modelo.named_steps["dropper"].get_feature_names_out(X_train.columns)
coefs = modelo.named_steps["linreg"].coef_

In [6]:
print(f"intercepto:{intercepto}")
print(f"feature_names:{feature_names}")
print(f"coefs:{coefs}")

intercepto:747.3873825211313
feature_names:['PC1' 'PC2' 'PC3' 'PC4' 'PC5' 'PC6' 'PC7' 'PC8' 'Gender___male'
 'Gender___other' 'City___ankara' 'City___antalya' 'City___bursa'
 'City___eskisehir' 'City___gaziantep' 'City___istanbul' 'City___izmir'
 'City___kayseri' 'City___konya' 'Product_Category___books'
 'Product_Category___electronics' 'Product_Category___fashion'
 'Product_Category___food' 'Product_Category___home & garden'
 'Product_Category___sports' 'Product_Category___toys'
 'Payment_Method___cash on delivery' 'Payment_Method___credit card'
 'Payment_Method___debit card' 'Payment_Method___digital wallet'
 'Device_Type___mobile' 'Device_Type___tablet'
 'Is_Returning_Customer___True']
coefs:[1016.97140689  -55.52373446  -62.73424174 -149.88939542  370.33389273
  -61.70843042   29.55556366 -784.72728358   49.5493102  -293.74155814
  106.31601827  162.45339521   85.31422002  -41.0174268   122.50277975
  188.90836945   71.61817115   23.92305266   69.46487228   49.34678424
  717.10090

In [7]:
# === Betas en espacio post-`pre` (antes del StandardScaler de la PCA) ===

# 1) Cargar el pipeline numérico usado en PCA_full
########################################################################################
pca_pipe = joblib.load("../01_preprocessing_results/preprocessing/pca_pipe_num.joblib")
########################################################################################
pre = pca_pipe.named_steps["pre"]  # ColumnTransformer (pre)
pre

,transformers,"[('num_med_rob', ...), ('num_med_min', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [8]:
scaler = pca_pipe.named_steps["std_for_pca"]  # StandardScaler
scaler

,copy,True
,with_mean,True
,with_std,True


In [9]:
pca = pca_pipe.named_steps["pca"]  # PCA(n_components=k)
pca

,n_components,8
,copy,True
,whiten,False
,svd_solver,'full'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,0


In [10]:
# 2) Extraer objetos necesarios
pre_feature_names = pre.get_feature_names_out()  # nombres de las p numéricas post-`pre`
pre_feature_names

array(['Discount_Amount', 'Unit_Price', 'Delivery_Time_Days',
       'Session_Duration_Minutes', 'Quantity', 'Age', 'Pages_Viewed',
       'Customer_Rating'], dtype=object)

In [11]:
mu = scaler.mean_  # (p,) Medias del EstandardScaler
mu

array([ 3.25682819e+00,  6.47624875e-01,  2.36892754e-01,  4.44211435e-02,
        2.47843137e-02, -3.58113539e-16, -5.23906844e-16,  4.33904764e-16])

In [12]:
print(sorted(mu,reverse=True))

[3.2568281934199748, 0.6476248747627688, 0.2368927536231884, 0.04442114347357066, 0.024784313725490194, 4.339047639708345e-16, -3.581135388230905e-16, -5.239068438337805e-16]


In [13]:
sig = scaler.scale_  # (p,) Desviaciones Estandar del StandardScaler
sig

array([11.86995217,  1.73442627,  0.14988097,  0.03726596,  0.03985189,
        1.        ,  1.        ,  1.        ])

In [14]:
print(sorted(sig))

[0.03726595843272831, 0.039851885200940594, 0.14988097261671002, 0.9999999999999999, 0.9999999999999999, 1.0, 1.734426269885633, 11.869952174906388]


In [15]:
C = pca.components_  # (k, p) Matriz de Componentes Principales (EigenVectores)
C

array([[ 0.70374816,  0.70287017, -0.03853837, -0.06313183,  0.04120118,
         0.01808745,  0.00440224,  0.0565433 ],
       [ 0.02174731,  0.06147511,  0.60912418,  0.34582238, -0.30441493,
        -0.57000222,  0.25460072,  0.15076466],
       [-0.04635716, -0.0437479 , -0.43314096, -0.26684814,  0.00309559,
        -0.2068985 ,  0.63262271,  0.54230069],
       [-0.01048287,  0.01977868, -0.15020186,  0.52509162, -0.41374221,
         0.43343739, -0.20621251,  0.5473965 ],
       [ 0.01494108, -0.02364008,  0.03035098,  0.54842154,  0.81149053,
         0.02945384,  0.14686442,  0.12875162],
       [ 0.02275672,  0.03282558,  0.03810233,  0.21602742, -0.21205665,
         0.46836489,  0.68030041, -0.47237976],
       [-0.04858098, -0.0073498 ,  0.64339246, -0.42664883,  0.17220154,
         0.47305118,  0.08939829,  0.37440612],
       [ 0.70633242, -0.70583204,  0.03136885, -0.00537248, -0.03610461,
         0.00920559,  0.00735812,  0.02098443]])

In [16]:
B = pca.explained_variance_  #Varianza Explicada por Valores Propios
B

array([1.41712957, 1.04007733, 1.02365781, 1.01209239, 0.99522832,
       0.97446018, 0.95292773, 0.58656057])

In [17]:
# 3) Aislar betas de Componentes Principales del modelo (según prefijo)
pc_prefix = "PC"
# Creación de la máscara para componentes principales
pc_mask = np.array([c.startswith(pc_prefix) for c in feature_names], dtype=bool)

# Extraccion de coeficientes de componentes principales
beta_PC = coefs[pc_mask]  # (k,)
 
# 4) Transformacion inversa: PCs -> post-`pre` (Variables Estandarizadas
gamma = C.T @ beta_PC
 
# Transformación: Variables estandarizadas → Variables originales
alpha_pre = gamma / sig  # betas por cada variable numérica post-`pre` (p,)

#Ajuste del Intercepto
beta0_pre = intercepto - np.sum(gamma * (mu / sig))  # intercepto en el mismo espacio
 
# 5) Empaquetar resultados
# Betas despues del Post Procesamiento
betas_post_pre_df = pd.DataFrame({
    "feature_post_pre": pre_feature_names,
    "beta_post_pre": alpha_pre
})
########################################################################################################
with open("../01_preprocessing_results/preprocessing/pca_metadata.json", "r", encoding="utf-8") as f:
#######################################################################################################
    meta = json.load(f)
 
orden_original_num = meta["cols_num"]  # orden original de columnas numéricas
presentes = set(betas_post_pre_df["feature_post_pre"].tolist())
orden_final = [c for c in orden_original_num if c in presentes]
 
betas_post_pre_df = (
    betas_post_pre_df
    .set_index("feature_post_pre")
    .loc[orden_final]
    .reset_index()
)
 
print("\nIntercepto (post `pre`):", beta0_pre)
print("\nBetas numéricas (post `pre`, en orden original):")
print(betas_post_pre_df)


Intercepto (post `pre`): -68.8791397576299

Betas numéricas (post `pre`, en orden original):
           feature_post_pre  beta_post_pre
0                       Age     -13.181867
1                Unit_Price     723.036505
2                  Quantity   11734.801560
3           Discount_Amount      14.101022
4  Session_Duration_Minutes     966.155031
5              Pages_Viewed      -9.160759
6        Delivery_Time_Days    -133.694358
7           Customer_Rating       4.491783


In [18]:
# === Predicción en TEST ===
yhat_test = modelo.predict(X_test)
 
# Métricas básicas en test
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
 
r2_test   = r2_score(y_test, yhat_test)
rmse_test = math.sqrt(mean_squared_error(y_test, yhat_test))
mae_test  = mean_absolute_error(y_test, yhat_test)
 
print({"R2_test": r2_test, "RMSE_test": rmse_test, "MAE_test": mae_test})
 
# También en train, para comparar generalización
yhat_train = modelo.predict(X_train)
r2_train   = r2_score(y_train, yhat_train)
rmse_train = math.sqrt(mean_squared_error(y_train, yhat_train))
mae_train  = mean_absolute_error(y_train, yhat_train)
print({"R2_train": r2_train, "RMSE_train": rmse_train, "MAE_train": mae_train})

{'R2_test': 0.5038629133578201, 'RMSE_test': 1371.6581931811868, 'MAE_test': 533.0901578558772}
{'R2_train': 0.5187867740988387, 'RMSE_train': 1510.8770918664284, 'MAE_train': 524.3997721900487}


In [19]:
# Veredicto

# NRMSE y mejora vs media en TEST
std_y_test = float(np.std(y_test, ddof=0))
nrmse_test = rmse_test / (std_y_test + 1e-12)
mejora_pct = 100.0 * (1.0 - nrmse_test)  # % mejor que predecir la media

# Semáforo (sin CV), "aquí dentro"
verde   = (r2_test >= 0.70) and (nrmse_test <= 0.50)
amarilo = (0.40 <= r2_test < 0.70) or (0.50 < nrmse_test <= 0.80)
if verde:
    veredicto = "VERDE"
    significado = "confiable para predecir aquí dentro."
elif amarilo:
    veredicto = "AMARILLO"
    significado = "usable con cautela (depende del caso de uso)"
else:
    veredicto = "ROJO"
    significado = "no confiable para predicción aquí dentro"

# Generar explicación humanizada automáticamente
def generar_explicacion(r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto):
    explicacion = []

    # Explicación basada en R²
    if r2_test >= 0.8:
        explicacion.append(f"• El modelo explica un {r2_test:.1%} de la variabilidad en los datos, lo que indica un excelente ajuste.")
        explicacion.append("  🎯 **Analogía**: Como un pronóstico del tiempo que acierta 8 de cada 10 días - muy confiable.")
    elif r2_test >= 0.6:
        explicacion.append(f"• El modelo captura un {r2_test:.1%} de la variabilidad, mostrando una buena capacidad predictiva.")
        explicacion.append("  🎯 **Analogía**: Similar a un forecast económico que identifica correctamente las tendencias principales.")
    elif r2_test >= 0.4:
        explicacion.append(f"• Con un R² del {r2_test:.1%}, el modelo tiene capacidad predictiva moderada.")
        explicacion.append("  🎯 **Analogía**: Como un detector de lluvia que funciona bien para saber si lloverá, pero no cuánto.")
    else:
        explicacion.append(f"• El R² de {r2_test:.1%} sugiere que el modelo tiene capacidad predictiva limitada.")
        explicacion.append("  🎯 **Analogía**: Parecido a adivinar el clima lanzando una moneda - mejor que nada, pero poco confiable.")

    # Explicación basada en NRMSE
    if nrmse_test <= 0.3:
        explicacion.append(f"• Los errores de predicción son muy bajos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como medir con una regla milimetrada - alta precisión en las estimaciones.")
    elif nrmse_test <= 0.5:
        explicacion.append(f"• Los errores son moderados ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Similar a usar una cinta métrica - útil para la mayoría de propósitos prácticos.")
    elif nrmse_test <= 0.7:
        explicacion.append(f"• Los errores son considerables ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como estimar distancias a ojo - sirve para aproximaciones gruesas.")
    else:
        explicacion.append(f"• Los errores son muy altos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Parecido a adivinar el tamaño de algo desde lejos - muy impreciso.")

    # Explicación basada en mejora vs media
    if mejora_pct > 50:
        explicacion.append(f"• Es un {mejora_pct:.0f}% mejor que simplemente predecir el promedio, una mejora sustancial.")
        explicacion.append("  🚀 **Analogía**: Como usar GPS vs. solo un mapa de carreteras - mucho más eficiente.")
    elif mejora_pct > 20:
        explicacion.append(f"• Mejora en un {mejora_pct:.0f}% respecto a predecir la media.")
        explicacion.append("  🚀 **Analogía**: Similar a tener indicaciones de tráfico en tiempo real - claramente mejor que sin ellas.")
    elif mejora_pct > 0:
        explicacion.append(f"• Solo un {mejora_pct:.0f}% mejor que predecir el promedio, mejora marginal.")
        explicacion.append("  🚀 **Analogía**: Como tener una brújula en lugar de solo el norte - ayuda, pero no demasiado.")
    else:
        explicacion.append("• No mejora respecto a predecir el valor promedio.")
        explicacion.append("  🚀 **Analogía**: Como intentar navegar sin brújula ni mapa - no aporta ventaja.")

    # Comparación entre train y test (si estuvieran disponibles ambos)
    if 'r2_train' in locals():
        sobreajuste = r2_train - r2_test
        if sobreajuste > 0.2:
            explicacion.append(f"• Hay indicios de sobreajuste (R² train: {r2_train:.3f} vs test: {r2_test:.3f}).")
            explicacion.append("  ⚠️ **Analogía**: Como un estudiante que memoriza las respuestas pero no entiende el concepto.")
        elif sobreajuste < 0.05:
            explicacion.append("• El modelo generaliza bien, sin signos evidentes de sobreajuste.")
            explicacion.append("  ✅ **Analogía**: Similar a un atleta que entrena y compite igual de bien - consistente.")

    # Interpretación del veredicto con analogías finales
    if veredicto == "VERDE":
        explicacion.append("\n✅ **Conclusión**: El modelo es confiable para hacer predicciones en contextos similares a los datos de prueba.")
        explicacion.append("🎯 **Analogía final**: Como un piloto automático confiable - puedes usarlo para navegar con seguridad.")
    elif veredicto == "AMARILLO":
        explicacion.append("\n⚠️ **Conclusión**: Úsalo con precaución - puede ser útil para identificar tendencias pero no para predicciones precisas.")
        explicacion.append("🎯 **Analogía final**: Como el forecast de fin de semana - útil para planear, pero lleva paraguas por si acaso.")
    else:
        explicacion.append("\n❌ **Conclusión**: Se recomienda revisar las variables o considerar modelos alternativos.")
        explicacion.append("🎯 **Analogía final**: Como un mapa muy antiguo - mejor conseguir uno actualizado o usar otros métodos.")

    return "\n".join(explicacion)

# Generar la explicación
explicacion_humanizada = generar_explicacion(
    r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto
)

resumen = {
    "R2_test": r2_test,
    "MAE_test": mae_test,
    "RMSE_test": rmse_test,
    "std(y_test)": std_y_test,
    "NRMSE_test": nrmse_test,
    "Mejora_vs_media_%": mejora_pct,
    "Veredicto": veredicto,
    "significado": significado
}

df_resumen = pd.DataFrame(resumen, index=[0]).T
df_resumen.columns = ["Valor"]
df_resumen.index.name = "Métrica"

print(df_resumen)
print("\n" + "="*60)
print("EXPLICACIÓN DEL VEREDICTO:")
print("="*60)
print(explicacion_humanizada)

def explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto):
    base = f"Con un R² de {r2_test:.3f} y un error relativo (NRMSE) de {nrmse_test:.3f}, "

    # Añadir analogías a la versión breve también
    if veredicto == "VERDE":
        analogia = "Como un GPS confiable - puedes seguir sus indicaciones con seguridad."
        return base + f"el modelo es robusto y explica bien los patrones en los datos, siendo {mejora_pct:.0f}% mejor que usar promedios simples. {analogia}"
    elif veredicto == "AMARILLO":
        analogia = "Similar a un pronóstico de lluvia - útil para planear, pero lleva sombrilla por si acaso."
        return base + f"el modelo tiene capacidad predictiva limitada ({mejora_pct:.0f}% mejor que promedios), adecuado para análisis exploratorios pero no para decisiones críticas. {analogia}"
    else:
        analogia = "Como un mapa desactualizado - mejor buscar herramientas más precisas."
        return base + f"el modelo no supera significativamente las predicciones básicas ({mejora_pct:.0f}% mejora), recomendando revisar el enfoque. {analogia}"

print("\n" + "🔍 RESUMEN INTERPRETATIVO:")
print("="*40)
print(explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto))

                                                          Valor
Métrica                                                        
R2_test                                                0.503863
MAE_test                                             533.090158
RMSE_test                                           1371.658193
std(y_test)                                         1947.354668
NRMSE_test                                              0.70437
Mejora_vs_media_%                                     29.563001
Veredicto                                              AMARILLO
significado        usable con cautela (depende del caso de uso)

EXPLICACIÓN DEL VEREDICTO:
• Con un R² del 50.4%, el modelo tiene capacidad predictiva moderada.
  🎯 **Analogía**: Como un detector de lluvia que funciona bien para saber si lloverá, pero no cuánto.
• Los errores son muy altos (70.4% de la variabilidad total).
  📏 **Analogía**: Parecido a adivinar el tamaño de algo desde lejos - muy impreciso.
• Mejora en u

In [20]:
###### Guardado del modelo
 
import joblib, json, time
 
# guarda el pipeline completo (dropper + LinearRegression)
joblib.dump(modelo, "modelo_reg_lineal.pkl")
 
# guarda el orden/esperado de columnas de entrenamiento
expected_cols = X_train.columns.tolist()
with open("expected_columns.json", "w", encoding="utf-8") as f:
    json.dump({"columns": expected_cols, "saved_at": time.strftime("%Y-%m-%d %H:%M:%S")}, f)
 
print("Artefactos guardados:", "modelo_reg_lineal.pkl", "expected_columns.json")

Artefactos guardados: modelo_reg_lineal.pkl expected_columns.json


In [21]:
import os, zipfile, glob
 
# Carpeta destino en tu PC
dst_dir = r"mi_regresion_lineal"
os.makedirs(dst_dir, exist_ok=True)
zip_path = os.path.join(dst_dir, "mi_reg_lin_artifacts_bundle.zip")
 
# Archivos que quieres incluir (ajusta si te falta alguno)
candidates = [
    "modelo_reg_lineal.pkl",
    "expected_columns.json",
]
 
present = [f for f in candidates if os.path.exists(f)]
# Si quieres incluir una carpeta (p. ej., 'sample_data'), descomenta:
# for root, _, files in os.walk("sample_data"):
#     for f in files:
#         present.append(os.path.join(root, f))
 
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for f in present:
        zf.write(f, arcname=os.path.basename(f))  # guarda sin subcarpetas
 
print("ZIP creado en:", zip_path)
print("Incluidos:", present)

ZIP creado en: mi_regresion_lineal\mi_reg_lin_artifacts_bundle.zip
Incluidos: ['modelo_reg_lineal.pkl', 'expected_columns.json']
